In [68]:
%%writefile queries.py
import MySQLdb

def testQuery(SGA):
    mydb = MySQLdb.connect("localhost", "root", "root", "TDI")
    cursor = mydb.cursor()
    query = "SELECT * FROM SigSGADEGTumor where driver_gene_name = '%s'"%(SGA)
    cursor.execute(query)
    result = cursor.fetchall()
    degs = []
    if len(result) == 0:
        return degs
    else:
        for row in result:
            degs.append(row[6])
        return degs


def getSGAtumorAll(SGA):
    '''
        get tumorset where SGA exists
    '''
    mydb = MySQLdb.connect("localhost", "root", "root", "TDI")
    cursor = mydb.cursor()
    query = " select S.patient_id from SGAs as S, Genes as G where S.gene_id = G.gene_id and G.gene_name = '%s'"%(SGA)
    cursor.execute(query)
    results = cursor.fetchall()
    tumorset = set()
    for result in results:
        tumorset.add(result[0])
    return tumorset
def getSGAtumordriver(SGA):
    '''
        get tumorset where SGA is driver
    '''
    mydb = MySQLdb.connect("localhost", "root", "root", "TDI")
    cursor = mydb.cursor()
    query = " select T.patient_id from TDI_driverCallPerTumor as T, Genes as G where T.gene_id = G.gene_id and G.gene_name = '%s'"%(SGA)
    cursor.execute(query)
    results = cursor.fetchall()
    tumorset = set()
    for result in results:
        tumorset.add(result[0])
    return tumorset

def getCT():
    '''
        get tumor types, save into disctionary
    '''
    mydb = MySQLdb.connect("localhost", "root", "root", "TDI")
    cursor = mydb.cursor()
    query = "select cancer_type_id, abbv from Cancer_Types"
    cursor.execute(query)
    results = cursor.fetchall()
    cancer_types = {}
    for result in results:
        #print result[0], result[1]
        cancer_types[result[0]] = result[1]
    return cancer_types

def tumorCTdistribution(tumors):
    '''
        get tumorset cancer type distribution
    '''
    mydb = MySQLdb.connect("localhost", "root", "root", "TDI")
    cursor = mydb.cursor()
    tumorset = '( '
    tumorset = '(' + ', '.join(str(tumor) for tumor in tumors) + ')'
    query = "select cancer_type_id, count(*) from Patients where patient_id in %s group by cancer_type_id"%(tumorset)
    cursor.execute(query)
    results = cursor.fetchall()
    return results

def getSGACTdistribution(SGA):
    '''
        get cancer type distribution and cancer types data
    '''
    tumorset_all = getSGAtumorAll(SGA)
    tumorset_driver = getSGAtumordriver(SGA)
    all_distribution = tumorCTdistribution(tumorset_all)
    driver_distribution = tumorCTdistribution(tumorset_driver)
    cancer_types = getCT()
    return (all_distribution, driver_distribution, cancer_types)

Overwriting queries.py


In [97]:
%%writefile Hello.py
from flask import Flask, render_template, request, json, url_for
from flask.ext.mysql import MySQL
from werkzeug import generate_password_hash, check_password_hash
#from flask_mysqldb import MySQL
from queries import *

app = Flask(__name__)
mysql = MySQL(app)
# MySQL configurations
app.config['MYSQL_DATABASE_USER'] = 'root'
app.config['MYSQL_DATABASE_PASSWORD'] = 'root'
app.config['MYSQL_DATABASE_DB'] = 'TDI'
app.config['MYSQL_DATABASE_HOST'] = 'localhost'
app.config['MYSQL_DATABASE_PORT'] = 3306
mysql.init_app(app)

@app.route('/test/')
def test():
    name = request.form['SGA']
    #return render_template('test.html', name = name)
    return render_template('test.html', name = name)
    
@app.route('/')
def main():
    (all_distribution, driver_distribution, cancer_types) = getSGACTdistribution('MUC6')
    all_distribution = list(int(i[1]) for i in all_distribution)
    driver_distribution = list(int(i[1]) for i in driver_distribution)
    return render_template('index.html', all_distribution = all_distribution, driver_distribution = driver_distribution, cancer_types = cancer_types)
    
    #return render_template('index.html')


@app.route('/showSignUp')
def showSignUp():
    return render_template('signup.html')


@app.route('/test/', methods=['POST', 'GET'])
def result():
    name = str(request.form.get('SGA', None))
    (all_distribution, driver_distribution, cancer_types) = getSGACTdistribution(name)
    all_distribution = list(int(i[1]) for i in all_distribution)
    driver_distribution = list(int(i[1]) for i in driver_distribution)
    return render_template('result.html', all_distribution = all_distribution, driver_distribution = driver_distribution, cancer_types = cancer_types)
    #return render_template('result.html', name = name)
if __name__ == '__main__':
    app.run()


Overwriting Hello.py


In [66]:
print len(A)
print len(B)

1908
1908


In [28]:
cancer_types[1]

'BLCA'

In [10]:
A = getSGACTAll('TP53')
B = getSGACTdriver('TP53')

In [45]:
tumors = set([1,2,3,4,5,6,7,8,9])
tumorset = ', '.join(str(tumor) for tumor in tumors)

In [46]:
tumorset

'1, 2, 3, 4, 5, 6, 7, 8, 9'